# Orchestrate Multiple-Agents for High-Quality Research with AutoGen and LangChain Tools

This notebook showcases how to harness the combined power of **AutoGen** and **LangChain** tools to automate and elevate deep research workflows. At its core, the system coordinates a network of specialized agents—each executing a distinct role in the research and report generation workflow. Together, these agents collect data, analyze findings, and produce polished, insight-driven reports.


[Open in Colab](https://colab.research.google.com/github/miztiik/taars/blob/master/notebooks/deepresearch_w_autogen_langchain_tools.ipynb) <a href="https://colab.research.google.com/github/miztiik/taars/blob/master/notebooks/deepresearch_w_autogen_langchain_tools.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🧠 Agent Roles Overview

- **🧭 Planner Agent**  
  Defines the research scope, objectives, and key questions.  
  → Anchors the process by structuring it around a well-defined `__TASK_QUERY`.

- **🔍 Researcher Agent**  
  Gathers information using tools like `wiki_search` and `duckduckgo_search`.  
  → Supplies the system with relevant, factual data.

- **🧪 Critic Agent**  
  Reviews the research plan, gathered evidence, and drafted reports.  
  → Provides feedback to enforce analytical rigor and clarity.

- **✍️ Editor Agent**  
  Refines the final report's language, structure, and presentation.  
  → Ensures clarity, polish, and alignment with stakeholder expectations.


In [ ]:
%%capture --no-stderr

%pip install -qU ipykernel
%pip install -qU autogen-agentchat
%pip install -qU autogen-ext

%pip install -qU loguru

%pip install -qU langchain
%pip install -qU langchain-community
%pip install -qU wikipedia
%pip install -qU lxml
%pip install -qU ddgs


In [ ]:
# %load_ext autoreload
# %autoreload 2
# %aimport -langchain_community
# Automatically reload modules before executing code

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html


In [ ]:
from pathlib import Path
from loguru import logger
import asyncio
import sys

# import logging
# logger = logging.getLogger("")


# # Remove all existing handlers
# for handler in logger.handlers[:]:
#     logger.removeHandler(handler)

# REMOVE ALL EXISTING HANDLERS for LOGURU
# if len(logger._core.handlers) > 1:
#     logger.configure(handlers=[])

# logger.add(
#     sys.stderr,
#     format="{time:HH:mm:ss:SSS} | {level} | {name}:{line} | {message}",
#     level="INFO",
#     colorize=True,
# )

# Configure file logging with better settings

# notebook_dir = Path.cwd()
# log_file = notebook_dir / "logs" / "llm_execution.log"
# log_file.parent.mkdir(exist_ok=True)
# logger.add(
#     log_file,
#     format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level: <8} | {name}:{function}:{line} | {message}",
#     level="DEBUG",
#     rotation="10 MB",
#     retention="7 days",
#     compression="zip",
#     enqueue=True,
# )
logger.info(f"✅ Logging configured successfully")


In [ ]:
import os
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo


# Confirm the API key is set
assert os.environ["GEMINI_API_KEY"], "GEMINI_API_KEY is not set"
assert os.environ["GEMINI_MODEL_NAME"], "GEMINI_MODEL_NAME is not set"
assert os.environ["GEMINI_BASE_URL"], "GEMINI_BASE_URL is not set"
assert os.environ["RATE_LIMIT_GEMINI_RPM"], "RATE_LIMIT_GEMINI_RPM is not set"
assert os.environ["RATE_LIMIT_GEMINI_RPD"], "RATE_LIMIT_GEMINI_RPD is not set"


model_info = ModelInfo(
    vision=True,
    function_calling=True,
    json_output=True,
    family=os.environ["GEMINI_MODEL_NAME"],
    structured_output=True,
)

model_client = OpenAIChatCompletionClient(
    model=os.environ["GEMINI_MODEL_NAME"],
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=os.environ["GEMINI_BASE_URL"],
    model_info=model_info,
)


In [ ]:
import asyncio, json, os
import tenacity
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.messages import StopMessage
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from ddgs import DDGS


In [ ]:
## Agent Tools

from pathlib import Path
from datetime import datetime
import re
from typing import Dict, Any

# Wiki Search Tool
wiki_api = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=2000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)


def wiki_search(q: str) -> dict:
    """Return structured output including text and source."""
    try:
        result = wiki_tool.run(q)
        return {"text": result, "source": "Wikipedia"}
    except Exception as e:
        return {"text": f"Error: {str(e)}", "source": None}

# DuckDuckGo Search Tool
def duckduckgo_search(q: str) -> dict:
    """Search the web using DuckDuckGo for information."""
    try:
        results = DDGS().text(q, region="us-en", safesearch="off", max_results=12)
        return {
            "text": results,
            "source": "DuckDuckGo",
            "query": q,
            "results_found": len(results),
        }
    except Exception as e:
        return {"text": f"Error: {str(e)}", "source": None}

def save_report(content: str, task_description: str, reports_dir: str = "reports") -> Dict[str, Any]:
    """
    Save a report to disk with automatic filename generation.
    
    This tool creates a timestamped Markdown file in the reports directory.
    Perfect for preserving research findings, analysis results, or final reports.
    
    Args:
        content (str): The report content to save. Can be plain text or Markdown.
                      If no title (# header) is present, one will be auto-generated.
        task_description (str): Brief description of the task/topic for filename.
                               Example: "AI impact on power sector analysis"
        reports_dir (str, optional): Directory to save reports. Defaults to "reports".
                                   Directory will be created if it doesn't exist.
    
    Returns:
        Dict[str, Any]: Result dictionary containing:
            - status: "success" or "error"
            - filepath: Full path to saved file (if successful)
            - filename: Just the filename (if successful)
            - error: Error message (if failed)
    
    Example Usage for Agents:
        # Save research findings
        result = save_report(
            content="# Market Analysis\\n\\nKey findings: ...",
            task_description="cryptocurrency market trends 2024"
        )
        
        # Save final report
        result = save_report(
            content=my_report_text,
            task_description="climate change impact assessment"
        )
        
        # Check if save was successful
        if result["status"] == "success":
            print(f"Report saved as: {result['filename']}")
        else:
            print(f"Save failed: {result['error']}")
    
    Generated Filename Format:
        YYYYMMDD_HHMM_meaningful_task_words.md
        Example: 20250819_1430_ai_power_sector.md
    
    Note: If file exists, automatic numbering prevents overwrites:
          20250819_1430_ai_power_sector_1.md, _2.md, etc.
    """
    try:
        # Ensure reports directory exists
        reports_path = Path(reports_dir)
        reports_path.mkdir(exist_ok=True)
        
        # Generate timestamped filename
        timestamp = datetime.now()
        date_time = timestamp.strftime("%Y%m%d_%H%M")
        task_name = _extract_task_name(task_description)
        
        filename = f"{date_time}_{task_name}.md"
        filepath = reports_path / filename
        
        # Handle filename conflicts with counter
        counter = 1
        while filepath.exists():
            filename = f"{date_time}_{task_name}_{counter}.md"
            filepath = reports_path / filename
            counter += 1
        
        # Format content with title if needed
        formatted_content = _format_content(content, task_description, timestamp)
        
        # Save to disk
        filepath.write_text(formatted_content, encoding='utf-8')
        logger.info(f"📄 Report saved: {filepath}")
        
        return {
            "status": "success",
            "filepath": str(filepath),
            "filename": filename,
            "timestamp": timestamp.isoformat()
        }
        
    except Exception as e:
        error_msg = f"Failed to save report: {str(e)}"
        logger.error(f"❌ {error_msg}")
        return {
            "status": "error",
            "error": error_msg,
            "task": task_description
        }


def _extract_task_name(task: str) -> str:
    """
    Extract 2-3 meaningful words from task description for filename.
    
    Args:
        task: The task description string
        
    Returns:
        Underscore-separated words suitable for filename
    """
    # Clean special characters and normalize
    clean_task = re.sub(r'[^\w\s]', ' ', task.lower())
    words = [w for w in clean_task.split() if len(w) > 2]
    
    # Filter common stop words
    stop_words = {
        'the', 'and', 'for', 'with', 'from', 'about', 'into', 'through',
        'during', 'before', 'after', 'above', 'below', 'over', 'under'
    }
    meaningful = [w for w in words if w not in stop_words][:3]
    
    return '_'.join(meaningful) if meaningful else 'report'


def _format_content(content: str, task: str, timestamp: datetime) -> str:
    """
    Format content with title and timestamp if needed.
    
    Args:
        content: Raw content to format
        task: Task description for title generation
        timestamp: When the report was created
        
    Returns:
        Formatted Markdown content
    """
    # If content already has a Markdown title, use as-is
    if content.strip().startswith('#'):
        return content
    
    # Add title and timestamp for plain text content
    formatted_title = f"# Report: {task}"
    timestamp_line = f"*Generated: {timestamp.strftime('%Y-%m-%d %H:%M')}*"
    
    return f"{formatted_title}\n\n{timestamp_line}\n\n{content}"


In [ ]:
## System Prompts for Agents

PLANNER_SYSTEM_PROMPT = """You create detailed research plans and handle replanning when obstacles arise.

WORKFLOW:
1. Draft detailed plan with steps, tools, and success criteria
2. Request Critic review: "Critic, please review this plan"
3. Revise based on feedback
4. Declare "PLAN_COMPLETE" only after Critic approval

REPLANNING (when research fails):
1. Analyze what failed and why
2. Pivot strategy with new keywords/approach
3. Request Critic review of revised plan
4. Declare "REVISED_PLAN_COMPLETE" after approval

Include: objectives, search strategies, tools to use, fallback approaches."""

RESEARCHER_SYSTEM_PROMPT = """Senior Research Analyst. Gather comprehensive evidence using strategic tool selection.

TOOLS:
- wiki_search: Background, definitions, historical context
- duckduckgo_search: Current events, market data, recent reports

EXECUTION:
1. Foundation research (wiki_search)
2. Current intelligence (duckduckgo_search)
3. Cross-validation

OUTPUT FORMAT:
# Evidence Summary: [Topic]
## Key Findings
## Current Intelligence
## Source Assessment
## Research Gaps

Focus on evidence quality, not presentation. Hand off to Editor for final reports."""

CRITIC_SYSTEM_PROMPT = """Domain-agnostic Critic. Provide concise, evidence-based critique.

FORMAT:
1. ARTIFACT: Type and goal (1 line)
2. STRENGTHS: Up to 3 key strengths
3. ISSUES: Critical/Major/Minor with evidence and fixes
4. VERDICT: Accept/Revise/Reject + confidence
5. VERACITY: Source verification requests

Respond "APPROVED: [summary]" when satisfied. Max 300 words total."""

EDITOR_SYSTEM_PROMPT = """Editorial Agent. Transform evidence into polished reports.

STANDARDS:
- Synthesis over summary
- Quantify impact with metrics
- Professional structure and tone
- Proper citations [1], [2]
- Use save_report tool for final output

TERMINATION: After save: "REPORT_SAVED. TASK_COMPLETED. TERMINATE"

Adapt structure to content type. Focus on actionable insights."""


In [ ]:
# assert os.environ["OPENAI_API_KEY"], "OPENAI_API_KEY is not set"
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"]
# )

# ---------- Agents ----------
from regex import R


planner = AssistantAgent(
    name="Planner",
    model_client=model_client,
    system_message=PLANNER_SYSTEM_PROMPT,
    description="Creates and adapts research plans, handles replanning when research hits obstacles",
)

researcher = AssistantAgent(
    name="Researcher",
    description="Expert research agent that strategically uses multiple tools to gather comprehensive, factual evidence and produces evidence-based draft reports.",
    model_client=model_client,
    tools=[wiki_search, duckduckgo_search],
    system_message=RESEARCHER_SYSTEM_PROMPT
)

critic = AssistantAgent(
    name="Critic",
    model_client=model_client,
    system_message=CRITIC_SYSTEM_PROMPT,
    description="Reviews and provides constructive criticism; outputs 'APPROVED: [...]' when ready.",
)


editor = AssistantAgent(
    name="Editor",
    description="Formats approved drafts with proper citations, adapts structure to content type.",
    model_client=model_client,
    tools=[save_report],
    system_message=EDITOR_SYSTEM_PROMPT,
)

# ---------- Selector prompt with few-shot examples ----------
selector_prompt = """You are the selector agent. Choose exactly one agent name from {participants} to speak next based on the conversation history and their roles.

Few-shot examples:

Example 1:
History: Researcher produced a draft; Critic says 'Needs more sources and clarity'.
Selected agent: Researcher

Example 2:
History: Researcher produced a draft; Critic says 'APPROVED'.
Selected agent: Editor

Roles:
{roles}

Conversation history:
{history}

Rules:
- Pick the agent whose role most advances completion.

Return only the agent name (no extra text).
"""


# ---------- Termination callable ----------
max_messages = 30
txt_termination = TextMentionTermination("TERMINATE")
termination_condition = (
    MaxMessageTermination(max_messages=max_messages) | txt_termination
)

# ---------- Build SelectorGroupChat ----------
team = SelectorGroupChat(
    participants=[planner, researcher, critic, editor],
    model_client=model_client,
    selector_prompt=selector_prompt,
    termination_condition=termination_condition,
)


In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
# @tenacity.retry(
#     wait=tenacity.wait_exponential(multiplier=1, min=60, max=120),
#     stop=tenacity.stop_after_attempt(2),
#     retry=tenacity.retry_if_exception_type(Exception),
# )
async def run_task(task_text: str):
    """
    Execute a multi-agent research task with proper termination handling.

    Args:
        task_text (str): The research task description

    Returns:
        str | None: Final report content or None if task incomplete
    """
    final_report = None
    task_completed = False

    try:
        logger.info(f"🚀 Starting task: {task_text}\n\n")

        async for event in team.run_stream(task=task_text):
            logger.info(f"📝 Event: {event}\n\n")

            # Check for TERMINATE keyword in message content
            if hasattr(event, "content") and isinstance(event.content, str):
                if "TERMINATE" in event.content:
                    final_report = event.content.split("TERMINATE", 1)[0].strip()
                    task_completed = True
                    logger.info("\n\n\n\n✅ Task completed with TERMINATE signal\n\n\n\n")
                    break


            # Simple delay between each agent call
            logger.info("\n\n\n\n⏳ Waiting to avoid rate limits...")
            await asyncio.sleep(25)

        return final_report if task_completed else None

    except Exception as e:
        logger.error(f"\n\n❌ Exception during task execution: {e}\n\n")
        raise

    finally:
        # Save final state on completion
        await _save_team_state()


async def _save_team_state():
    """Helper function to save team state with error handling."""
    try:
        state = await team.save_state()
        with open("team_state.json", "w") as f:
            json.dump(state, f, indent=2)
        logger.info("💾 Team state saved successfully")
    except Exception as e:
        logger.error(f"❌ Failed to save team state: {e}")


async def resume_from_saved_state():
    """Resume execution from previously saved team state."""
    try:
        with open("team_state.json", "r") as f:
            saved_state = json.load(f)

        await team.load_state(saved_state)
        logger.info("🔄 Resuming from saved state")

        async for event in team.run_stream():
            logger.info(f"📝 Resume Event: {event}")

    except FileNotFoundError:
        logger.error("❌ No saved state file found")
    except Exception as e:
        logger.error(f"❌ Failed to resume from saved state: {e}")
        raise


In [ ]:
if __name__ == "__main__":

    __SAMPLE_QUERY_1 = "Indian steel sector growth post modernization and growth prospects in an era of US tariffs and reduce government protection through trade barriers and cheaper import options from China"

    __SAMPLE_QUERY_2 = "How government and governance factors improved economy and lives of indians during Modi and Pre-Modi starting from 1991"

    __SAMPLE_QUERY_3 = "Sectoral growth based on cyclics for 2025 and macro economic pressure and trade tariffs and uncertainty, which sectors are best poised for maximum investment returns in terms of % for the next year for a moderate to average risk profile investments"

    __SAMPLE_QUERY_4 = "Hyperscaler investments in data centers and cloud infrastructure for AI growth is not matching the proposed productivity gains in GDP. Are we witnessing a bubble? Is % of global GDP being invested in AI infrastructure matches the productivity gain percentages?"

    __SAMPLE_QUERY_5 = "If neural networks are foundation of LLMs and based on the human brain; Are LLMs given tools during training? Humans learn with tool usage, What are current trends on tool usage in LLM training?"

    try:
        output = asyncio.run(run_task(__SAMPLE_QUERY_1))
        if output:
            print("\n" + "=" * 60)
            print("🎯 FINAL REPORT")
            print("=" * 60)
            print(output)
        else:
            print("❌ Task did not complete successfully.")
    except Exception as e:
        logger.error(f"❌ Fatal error: {e}")
        print(f"❌ Execution failed: {e}")
